In [12]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
import os
import re
from pyextremes import EVA

def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

def list_dates(start_date_str, end_date_str):
    # Convert the start and end date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y%m%d")
    end_date = datetime.strptime(end_date_str, "%Y%m%d")

    # Initialize an empty list to store the dates
    date_list = []

    # Create a loop to generate the list of dates
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime("%Y%m%d"))
        current_date += timedelta(days=1)
    
    return date_list

# Data

In [122]:
district_df = pd.read_csv('data/district.csv')
district_cols = ['bps_kota_nama', 'bps_kecamatan_nama', 'bps_kecamatan_kode', 'latitude', 'longitude']
start_date = '20220101'
end_date = '20230630'
date_list = list_dates(start_date, end_date)

district_date_df = district_df.drop_duplicates().dropna(subset=district_cols[1:])[district_cols].copy()
id_to_date = {unique_id: date_list for unique_id in district_date_df['bps_kecamatan_kode']}
district_date_df['date'] = district_date_df['bps_kecamatan_kode'].map(id_to_date)
district_date_df = district_date_df.explode('date')

district_date_df.head()

,bps_kota_nama,bps_kecamatan_nama,bps_kecamatan_kode,latitude,longitude,date
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,20220101
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,20220102
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,20220103
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,20220104
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,20220105


In [18]:
data = district_date_df.copy()
feat_files = [f for f in os.listdir('./data') if '_tab_all.csv' in f]

for file in feat_files:
    fe = re.search("^(\w+)_tab", file).group(1)
    print(f'processing data {fe}...')
    
    feat_df = pd.read_csv(f"data/{file}")
    feat_df['date'] = feat_df['date'].astype(str) 
    feat_df.insert(loc=0, column='lat_long_id', value=feat_df.set_index(['latitude','longitude']).index.factorize()[0]+1)
    
    lat_long_feat = feat_df[['lat_long_id', 'latitude', 'longitude']].drop_duplicates().copy()
    kec_df = district_date_df[district_cols].drop_duplicates().copy()

    kec_feat = kec_df.merge(lat_long_feat, how='cross', suffixes=('', '_feat'))
    kec_feat['distance'] = kec_feat.apply(lambda x: haversine(x['latitude'], x['longitude'], x['latitude_feat'], x['longitude_feat']), axis=1)
    kec_feat['rnk'] = kec_feat.groupby(['bps_kecamatan_kode'])['distance'].rank(method='dense')

    data_per_feat = kec_feat[kec_feat['rnk'] == 1][['bps_kecamatan_kode', 'lat_long_id']].merge(feat_df[['lat_long_id', 'date', 'value']], how='left', on='lat_long_id').drop('lat_long_id', axis=1)
    data_per_feat.rename(columns={'value':fe}, inplace=True)


    data = data.merge(data_per_feat, how='left', on=['bps_kecamatan_kode', 'date'])

data = data.groupby(district_cols, as_index=False).apply(lambda group: group.ffill())
data.head()

processing data humidity...
processing data precipitation...
processing data windspeed...
processing data temperature...
processing data shortwavenet...


bps_kota_nama bps_kecamatan_nama  bps_kecamatan_kode  latitude  \
0 70980  KABUPATEN BANDUNG           ARJASARI             3204150  -7.06233   
  70981  KABUPATEN BANDUNG           ARJASARI             3204150  -7.06233   
  70982  KABUPATEN BANDUNG           ARJASARI             3204150  -7.06233   
  70983  KABUPATEN BANDUNG           ARJASARI             3204150  -7.06233   
  70984  KABUPATEN BANDUNG           ARJASARI             3204150  -7.06233   

         longitude      date  humidity  precipitation  windspeed  temperature  \
0 70980  107.64064  20220101     84.50       2.628378   3.960024      32.4125   
  70981  107.64064  20220102     84.50       4.248128   3.960024      32.4125   
  70982  107.64064  20220103     56.25       0.396570   3.960024      32.3500   
  70983  107.64064  20220104     56.25       0.000000   3.960024      32.3500   
  70984  107.64064  20220105     70.50       0.089887   3.960024      30.1625   

         shortwavenet  
0 70980     171.56462  
  70981     198.82175  
  70982     245.18614  
  70983     230.97664  
  70984     211.58810

In [20]:
data.to_csv('data/district_feature.csv', index=False)

# Model

In [13]:
data = pd.read_csv('data/district_feature.csv')
# data['date'] = pd.to_datetime(data['date'].astype(str)).dt.strftime("%Y-%m-%d")

district_df = pd.read_csv('data/district.csv')
district_cols = ['bps_kota_nama', 'bps_kecamatan_nama', 'bps_kecamatan_kode', 'latitude', 'longitude']
start_date = '20220101'
end_date = '20231210'
date_list = list_dates(start_date, end_date)

district_date_df = district_df.drop_duplicates().dropna(subset=district_cols[1:])[district_cols].copy()
id_to_date = {unique_id: date_list for unique_id in district_date_df['bps_kecamatan_kode']}
district_date_df['date'] = district_date_df['bps_kecamatan_kode'].map(id_to_date)
district_date_df = district_date_df.explode('date')
district_date_df['date'] = pd.to_datetime(district_date_df['date']).dt.strftime("%Y-%m-%d")

district_date_df.head()

,bps_kota_nama,bps_kecamatan_nama,bps_kecamatan_kode,latitude,longitude,date
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,2022-01-01
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,2022-01-02
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,2022-01-03
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,2022-01-04
0,KABUPATEN BOGOR,CIBINONG,3201210,-6.47891,106.845,2022-01-05


In [14]:
start_date_str = "2023-07-02"
number_of_weeks = 24
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
list_of_weeks = []

for i in range(number_of_weeks):
    list_of_weeks.append(start_date.strftime("%Y-%m-%d"))
    start_date += timedelta(days=7)

In [24]:
full_df = district_date_df.copy()
clean_data = data[data['date'].astype(int) >= 20220102].copy()
clean_data['date'] = pd.to_datetime(clean_data['date'].astype(str))
clean_data['month'] = clean_data['date'].apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-01"))

cities_ = ['KABUPATEN INDRAMAYU', 'KABUPATEN KARAWANG', 'KABUPATEN SUBANG']
data_2022 = clean_data[(clean_data['date'] >= pd.to_datetime("2022-01-02")) 
                       & (clean_data['date'] < pd.to_datetime("2023-01-01"))].copy()
city_data = clean_data[clean_data['bps_kota_nama'].isin(cities_)].copy()

for feature_ in ['temperature', 'windspeed', 'shortwavenet', 'humidity', 'precipitation']:
    print(f'processing for {feature_}')
    summary_df = pd.DataFrame()
    for k in city_data['bps_kecamatan_kode'].unique():
        print(f'kecamatan_kode: {k}')
        subcity_data = city_data[city_data['bps_kecamatan_kode'] == k].copy()
        input_data = subcity_data.set_index('date')[feature_]

        model = EVA(data=input_data)
        model.get_extremes(
            method="BM",
            extremes_type="high",
            block_size="7D",
            errors="raise",
        )

        model.fit_model()
        summary = model.get_summary(
            return_period=range(1,25),
            alpha=0.95,
            n_samples=1000,
        )

        summary.index = list_of_weeks
        temp_sum_df = summary[['return value']].reset_index()
        temp_sum_df.columns = ['date', feature_]
        temp_sum_df['bps_kecamatan_kode'] = k
        temp_sum_df['month'] = temp_sum_df['date'].apply(lambda x: pd.to_datetime(x).strftime("%m"))

        summary_df = pd.concat([summary_df, temp_sum_df], ignore_index=True)
    
    xt_df = data_2022.groupby(['bps_kecamatan_kode', 'month'])[feature_].describe()[['25%', '75%']].reset_index()
    xt_df['month'] = xt_df['month'].apply(lambda x: pd.to_datetime(x).strftime("%m"))
    
    drop_cols = ['month', '25%', '75%','25_diff', '75_diff']
    detail_district_df = district_df[['bps_kecamatan_kode','latitude','longitude']].dropna()

    res_df = summary_df.merge(xt_df, how = 'left', on=['bps_kecamatan_kode', 'month'])
    res_df['25_diff'] = res_df[feature_] - res_df['25%']
    res_df['75_diff'] = res_df[feature_] - res_df['75%']

    res_df[f'{feature_}_extreme_perc'] = np.where(res_df['25_diff'] < 0, round(res_df['25_diff']*100/res_df[feature_], 2), 
                                                  np.where(res_df['75_diff'] > 0, round(res_df['75_diff']*100/res_df[feature_], 2), 0))

    res_df[f'is_{feature_}_extreme'] = np.where(res_df[f'{feature_}_extreme_perc'] != 0, True, False)

#     feat_df = district_feature[['bps_kecamatan_kode', 'date', feature_]].copy()
#     res_feat_df = pd.concat([feat_df, res_df.drop(drop_cols, axis=1)], axis=0)
    full_df = full_df.merge(res_df.drop(drop_cols, axis=1), on = ['bps_kecamatan_kode', 'date'])

print(full_df.shape)
full_df.head()

processing for temperature
kecamatan_kode: 3212210
kecamatan_kode: 3212171
kecamatan_kode: 3212140
kecamatan_kode: 3212060
kecamatan_kode: 3212200
kecamatan_kode: 3212161
kecamatan_kode: 3212040
kecamatan_kode: 3212030
kecamatan_kode: 3212011
kecamatan_kode: 3212010
kecamatan_kode: 3212150
kecamatan_kode: 3212130
kecamatan_kode: 3212110
kecamatan_kode: 3212190
kecamatan_kode: 3212100
kecamatan_kode: 3212101
kecamatan_kode: 3212080
kecamatan_kode: 3212090
kecamatan_kode: 3212020
kecamatan_kode: 3212050
kecamatan_kode: 3212170
kecamatan_kode: 3212180
kecamatan_kode: 3212162
kecamatan_kode: 3212221
kecamatan_kode: 3212160
kecamatan_kode: 3212120
kecamatan_kode: 3212081
kecamatan_kode: 3212220
kecamatan_kode: 3212041
kecamatan_kode: 3212061
kecamatan_kode: 3212070
kecamatan_kode: 3215071
kecamatan_kode: 3215190


/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.p

kecamatan_kode: 3215020


/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)
/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.p

kecamatan_kode: 3215170


...

/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)


kecamatan_kode: 3215010
kecamatan_kode: 3215160
kecamatan_kode: 3215051
kecamatan_kode: 3215120
kecamatan_kode: 3215150
kecamatan_kode: 3215100
kecamatan_kode: 3215011
kecamatan_kode: 3215032
kecamatan_kode: 3215031


/home/figarrik/.virtualenvs/wrenv/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:2500: RuntimeWarning: invalid value encountered in subtract
  -pex2+logpex2-logex2)


kecamatan_kode: 3215130
kecamatan_kode: 3215180
kecamatan_kode: 3215060
kecamatan_kode: 3213170
kecamatan_kode: 3213220
kecamatan_kode: 3213180
kecamatan_kode: 3213021
kecamatan_kode: 3213060
kecamatan_kode: 3213050
kecamatan_kode: 3213130
kecamatan_kode: 3213090
kecamatan_kode: 3213150
kecamatan_kode: 3213030
kecamatan_kode: 3213160
kecamatan_kode: 3213081
kecamatan_kode: 3213020
kecamatan_kode: 3213080
kecamatan_kode: 3213031
kecamatan_kode: 3213210
kecamatan_kode: 3213100
kecamatan_kode: 3213140
kecamatan_kode: 3213141
kecamatan_kode: 3213190
kecamatan_kode: 3213110
kecamatan_kode: 3213120
kecamatan_kode: 3213201
kecamatan_kode: 3213200
kecamatan_kode: 3213010
kecamatan_kode: 3213011
kecamatan_kode: 3213070
kecamatan_kode: 3213191
kecamatan_kode: 3213171
kecamatan_kode: 3213040
(2184, 21)


,bps_kota_nama,bps_kecamatan_nama,bps_kecamatan_kode,latitude,longitude,date,temperature,temperature_extreme_perc,is_temperature_extreme,windspeed,...,is_windspeed_extreme,shortwavenet,shortwavenet_extreme_perc,is_shortwavenet_extreme,humidity,humidity_extreme_perc,is_humidity_extreme,precipitation,precipitation_extreme_perc,is_precipitation_extreme
0,KABUPATEN INDRAMAYU,HAURGEULIS,3212010,-6.46413,107.94734,2023-07-02,35.437392,11.36,True,5.633499,...,True,206.022376,20.39,True,90.715134,14.57,True,101.093336,99.01,True
1,KABUPATEN INDRAMAYU,HAURGEULIS,3212010,-6.46413,107.94734,2023-07-09,35.708961,12.03,True,5.802149,...,True,207.382036,20.92,True,91.922845,15.69,True,128.185565,99.22,True
2,KABUPATEN INDRAMAYU,HAURGEULIS,3212010,-6.46413,107.94734,2023-07-16,35.842903,12.36,True,5.886095,...,True,207.990918,21.15,True,92.559448,16.27,True,146.369568,99.31,True
3,KABUPATEN INDRAMAYU,HAURGEULIS,3212010,-6.46413,107.94734,2023-07-23,35.928360,12.57,True,5.939962,...,True,208.356733,21.29,True,92.982848,16.65,True,160.452487,99.37,True
4,KABUPATEN INDRAMAYU,HAURGEULIS,3212010,-6.46413,107.94734,2023-07-30,35.989681,12.72,True,5.978777,...,True,208.607901,21.38,True,93.296081,16.93,True,172.107351,99.42,True


In [25]:
full_df.to_csv('data/data_model.csv', index=False)